# Payee

In [3]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [4]:
import os
import json
import torch
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
TrOCRProcessor,
VisionEncoderDecoderModel,
Seq2SeqTrainer,
Seq2SeqTrainingArguments
)
from torchvision import transforms
from google.colab import drive
import shutil

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
CROPPED_DIR = "/content/drive/MyDrive/whited_payee"
LABELS_DIR = "/content"
MODEL_BASE_DIR = "/content/trained_models"
DRIVE_BACKUP_PATH = "/content/drive/MyDrive/CheckModelsBackup"
FIELD = "payee"

# Did u get the labels done?

In [8]:
label_path = os.path.join(LABELS_DIR, f"{FIELD}_labels.json")
with open(label_path, "r", encoding="utf-8") as f:
  annotations = json.load(f)

In [9]:
annotations = [
item for item in annotations
if os.path.exists(os.path.join(CROPPED_DIR, item["image"])) and item["text"].strip().lower() != "null"
]

In [10]:
train_items, val_items = train_test_split(annotations, test_size=0.2, random_state=42)

In [11]:
def build_dataset(items):

  dataset_dict = {"image": [], "text": []}
  for item in items:
    dataset_dict["image"].append(os.path.join(CROPPED_DIR, item["image"]))
    dataset_dict["text"].append(item["text"].strip())
  return Dataset.from_dict(dataset_dict)

train_dataset = build_dataset(train_items)
val_dataset = build_dataset(val_items)

In [12]:
#Load Processor and Model

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
augment = transforms.Compose([
transforms.RandomRotation(2),
transforms.ColorJitter(brightness=0.1, contrast=0.1)
])

In [15]:
def preprocess(example):
  image = Image.open(example["image"]).convert("RGB")
  image = augment(image)
  pixel_values = processor(image, return_tensors="pt").pixel_values.squeeze(0)
  input_ids = processor.tokenizer(
    example["text"],
    padding="max_length",
    max_length=64,
    truncation=True
  ).input_ids
  return {
  "pixel_values": pixel_values,
  "labels": torch.tensor(input_ids, dtype=torch.long)
  }

train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")

Map:   0%|          | 0/2716 [00:00<?, ? examples/s]

Map:   0%|          | 0/679 [00:00<?, ? examples/s]

In [20]:
def collate_fn(batch):
  pixel_values = torch.stack([item["pixel_values"] for item in batch])
  labels = torch.nn.utils.rnn.pad_sequence(
    [item["labels"] for item in batch],
    batch_first=True,
    padding_value=processor.tokenizer.pad_token_id
  )
  decoder_input_ids = labels[:, :-1].contiguous()
  labels_shifted = labels[:, 1:].clone()
  labels_shifted[labels_shifted == processor.tokenizer.pad_token_id] = -100
  return {
    "pixel_values": pixel_values,
    "decoder_input_ids": decoder_input_ids,
    "labels": labels_shifted
  }

# Im fed up of applying all the parameters--> amrit

# Amrit, the data for payee is not generalise ... i just plotted the graph .. thats why it is getting overfitted
# we cannot do any thing, we dont have that much of data from bank to generalise-->shivanshu

In [18]:
training_args = Seq2SeqTrainingArguments(
output_dir=os.path.join(MODEL_BASE_DIR, FIELD),
per_device_train_batch_size=8,
num_train_epochs=10,
learning_rate=3e-5,
save_strategy="no",
logging_dir=f"./logs/{FIELD}",
predict_with_generate=True,
fp16=True,
report_to="none",
label_smoothing_factor=0.1,
lr_scheduler_type="cosine",
warmup_ratio=0.1
)

trainer = Seq2SeqTrainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=val_dataset,
tokenizer=processor,
data_collator=collate_fn
)

<ipython-input-18-4e6a0a24ad09>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [19]:
trainer.train()

Step,Training Loss
500,2.841300
1000,2.314100
1500,1.906900
2000,1.706600
2500,1.575200
3000,1.510500


TrainOutput(global_step=3400, training_loss=1.9182895076976103, metrics={'train_runtime': 2156.882, 'train_samples_per_second': 12.592, 'train_steps_per_second': 1.576, 'total_flos': 2.032342578745049e+19, 'train_loss': 1.9182895076976103, 'epoch': 10.0})

In [21]:
model_path = os.path.join(MODEL_BASE_DIR, FIELD)
model.save_pretrained(model_path)
processor.save_pretrained(model_path)
print(f"Model saved temporarily at: {model_path}")

Model saved temporarily at: /content/trained_models/payee


In [22]:
drive_model_path = os.path.join(DRIVE_BACKUP_PATH, FIELD)
shutil.copytree(model_path, drive_model_path)
print(f"Final model saved to Google Drive: {drive_model_path}")

Final model saved to Google Drive: /content/drive/MyDrive/CheckModelsBackup/payee


In [23]:
def evaluate_on_dataset(dataset, split_name):
    correct = 0
    total = 0
    for example in tqdm(dataset, desc=f"Evaluating {FIELD} on {split_name}"):
        image_tensor = example['pixel_values'].unsqueeze(0).to(model.device)
        label_text = processor.tokenizer.decode(example['labels'], skip_special_tokens=True)
        with torch.no_grad():
            outputs = model.generate(
                image_tensor,
                max_length=64,
                num_beams=5,
                early_stopping=True
            )
        prediction = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        if prediction.strip().lower() == label_text.strip().lower():
            correct += 1
        total += 1
    accuracy = correct / total * 100
    print(f"{split_name} Accuracy for {FIELD}: {accuracy:.2f}%")

In [24]:
evaluate_on_dataset(train_dataset, "train")
evaluate_on_dataset(val_dataset, "validation")

🔍 Evaluating payee on train: 100%|██████████| 2716/2716 [11:22<00:00,  3.98it/s]


train Accuracy for payee: 98.82%


🔍 Evaluating payee on validation: 100%|██████████| 679/679 [02:52<00:00,  3.93it/s]

validation Accuracy for payee: 52.28%
